In [1]:
# Get the data for expeirment
import pandas as pd 
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import seaborn as sns
import TsModel
import GprModel
import TsModel_constant
import torch
from sklearn.metrics import r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers, models

device = torch.device("cuda:0")
print(f"Keras version is {keras.__version__}")
print(f"Num GPUs Available: {torch.cuda.device_count()}")
plt.rcParams.update({'font.size': 20})

Keras version is 3.1.1
Num GPUs Available: 1


In [19]:
exp28_results = {
    'gp_tsl_train_accuracy':gp_tsl_train_accuracy,
    'gp_tsl_test_accuracy':gp_tsl_test_accuracy,
    'gp_tsl_r2s':gp_tsl_r2s,
    'gp_tsl_time':gp_tsl_time,
    'gp_tsc_train_accuracy':gp_tsc_train_accuracy,
    'gp_tsc_test_accuracy':gp_tsc_test_accuracy,
    'gp_tsc_r2s':gp_tsc_r2s,
    'gp_tsc_time':gp_tsc_time,
    'gp_nn_tsl_train_accuracy':gp_nn_tsl_train_accuracy,
    'gp_nn_tsl_test_accuracy':gp_nn_tsl_test_accuracy,
    'gp_nn_tsl_r2s':gp_nn_tsl_r2s,
    'nn_tsl_r2s':nn_tsl_r2s,
    'gp_nn_tsl_time':gp_nn_tsl_time,
    'gp_nn_tsc_train_accuracy':gp_nn_tsc_train_accuracy,
    'gp_nn_tsc_test_accuracy':gp_nn_tsc_test_accuracy,
    'gp_nn_tsc_r2s':gp_nn_tsc_r2s,
    'nn_tsc_r2s':nn_tsc_r2s,
    'gp_nn_tsc_time':gp_nn_tsc_time
}

In [20]:
#%store exp28_results 

Stored 'exp28_results' (dict)


In [2]:
# Train and test the solar data set
x_train = pd.read_csv('Data/FES/processed_solar_radiation_x_train.csv')
x_test = pd.read_csv('Data/FES/processed_solar_radiation_x_test.csv')
y_train = pd.read_csv('Data/FES/processed_solar_radiation_y_train.csv')
y_test = pd.read_csv('Data/FES/processed_solar_radiation_y_test.csv')

In [3]:
number_of_folds = 10

# Source: From the TSmodel
def preprocess_data(train_x, test_x, train_y, test_y):
    feature_scaler = StandardScaler()
    scaled_train_x = feature_scaler.fit_transform(train_x)
    scaled_test_x = feature_scaler.transform(test_x)

    # target_scaler = StandardScaler()  
    # scaled_train_y = target_scaler.fit_transform(train_y)
    # scaled_test_y = target_scaler.transform(test_y)    
    # return scaled_train_x, scaled_test_x, scaled_train_y, scaled_test_y, feature_scaler, target_scaler    

    return scaled_train_x, scaled_test_x, feature_scaler    

def build_neural_network_model():
    model = models.Sequential()
    model.add(layers.Dense(7, activation=keras.activations.tanh))    
    model.add(layers.Dense(5, activation=keras.activations.tanh))    
    model.add(layers.Dense(1, activation=keras.activations.relu))
    return model

In [4]:
x_train, x_test, feature_scaler = preprocess_data(x_train, x_test, y_train, y_test)
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [14]:
def do_nn_exp():
    neural_network_model = build_neural_network_model()

    neural_network_model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.MeanSquaredError(),
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    start_time = time.time()
    history = neural_network_model.fit(
        x_train, 
        y_train,
        epochs=100, 
        shuffle=True
    )
    time_used = time.time() - start_time
    
    return neural_network_model

In [15]:
nn_model = do_nn_exp()

Epoch 1/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 142388.4688 - root_mean_squared_error: 377.2706
Epoch 2/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 137196.0625 - root_mean_squared_error: 370.3521
Epoch 3/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 139864.3281 - root_mean_squared_error: 373.9633
Epoch 4/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 139367.7500 - root_mean_squared_error: 373.2242
Epoch 5/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 138164.8125 - root_mean_squared_error: 371.6081
Epoch 6/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 135974.2031 - root_mean_squared_error: 368.7109
Epoch 7/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 132213.9219 - root_mean_squared_error: 363.4794
Epoch 8/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 128496.1016 - root_mean_squared_error: 358.4426
Epoch 9/100
715/715 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 130348.9766 - root_mean_squared_error: 360.8931
Epoch 10/100
715/71

In [16]:
# TS model
# Linear conclusion

def ts_relu_wrapper(pred):
    return np.maximum(0, pred)

# TS with linear conclusion
tsl_model = TsModel.TsModel(number_of_rules=25, fuzzification_coefficient=1.2)

start_time = time.time()
tsl_model.fit(x_train, y_train)
time_used = time.time() - start_time

FCM training RMSE: 142.78836944251145


In [10]:
# TS model
# Constant conclusion


tsc_model = TsModel_constant.TsModel_constant(number_of_rules=25, fuzzification_coefficient=1.2)

start_time = time.time()
tsc_model.fit(x_train, y_train)
time_used = time.time() - start_time

(22880, 200)
[[6.43845679e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.52240574e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.94583407e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.79532449e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.36205127e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.35219519e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(22880, 25)
[[6.43845679e-06 1.92210945e-06 3.47948160e-06 ... 1.09903347e-05
  7.05354762e-06 6.94579570e-05]
 [1.52240574e-01 2.36120224e-05 5.68416618e-05 ... 5.21953531e-05
  4.70753400e-04 1.84639985e-04]
 [6.94583407e-05 2.81963685e-06 6.65581384e-06 ... 2.75596636e-05
  3.10804513e-05 9.35498166e-01]
 ...
 [1.79532449e-03 5.59622590e-05 6.79466771e-04 ... 1.89297176e-02
  9.071

In [11]:
# GPR model
gp_tsl_train_accuracy = []
gp_tsl_test_accuracy = []
gp_tsl_r2s = []
gp_tsl_time = []
gpr_tsl_model = None

# From ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)

    start_time = time.time()
    gpr_tsl_model.fit(tsl_model.cen, ts_relu_wrapper(tsl_model.predict(tsl_model.cen + 1e-8)))
    time_used = time.time() - start_time

    train_means = gpr_tsl_model.predict(x_train)
    test_means = gpr_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsl_train_accuracy.append(train_rmse)
    gp_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsl_r2s.append(gp_r2)
    gp_tsl_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsl_time),3)
gp_time_std = np.round(np.std(gp_tsl_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\

GP Train RMSE: 247.751 ± 86.065
GP Test RMSE: 245.381 ± 84.362
GP R2: 0.316 ± 0.484
GP Time: 0.024 ± 0.007


In [12]:
# GPR model
gp_tsc_train_accuracy = []
gp_tsc_test_accuracy = []
gp_tsc_r2s = []
gp_tsc_time = []
gpr_tsc_model = None

# From ts constant
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_tsc_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)

    start_time = time.time()
    gpr_tsc_model.fit(tsc_model.cen, ts_relu_wrapper(tsc_model.predict(tsc_model.cen + 1e-8)))
    time_used = time.time() - start_time

    train_means = gpr_tsc_model.predict(x_train)
    test_means = gpr_tsc_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_tsc_train_accuracy.append(train_rmse)
    gp_tsc_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_tsc_r2s.append(gp_r2)
    gp_tsc_time.append(time_used)

gp_train_accuracy_mean = np.round(np.mean(gp_tsc_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_tsc_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_tsc_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_tsc_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_tsc_r2s),3)
gp_r2_std = np.round(np.std(gp_tsc_r2s),3)
gp_time_mean = np.round(np.mean(gp_tsc_time),3)
gp_time_std = np.round(np.std(gp_tsc_time),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\

GP Train RMSE: 187.854 ± 0.0
GP Test RMSE: 185.655 ± 0.0
GP R2: 0.65 ± 0.0
GP Time: 0.025 ± 0.011


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\

In [17]:
# GPR model
gp_nn_tsl_train_accuracy = []
gp_nn_tsl_test_accuracy = []
gp_nn_tsl_r2s = []
nn_tsl_r2s = []
gp_nn_tsl_time = []
gpr_nn_tsl_model = None

# From NN and ts linear
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_nn_tsl_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)

    start_time = time.time()
    inputs = tsl_model.cen
    ts_predictions = ts_relu_wrapper(tsl_model.predict(tsl_model.cen + 1e-8))
    nn_predictions = nn_model.predict(inputs)
    gpr_nn_tsl_model.fit(inputs, nn_predictions)
    time_used = time.time() - start_time

    train_means = gpr_nn_tsl_model.predict(x_train)
    test_means = gpr_nn_tsl_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_nn_tsl_train_accuracy.append(train_rmse)
    gp_nn_tsl_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_nn_tsl_r2s.append(gp_r2)
    gp_nn_tsl_time.append(time_used)
    nn_tsl_r2s.append(r2_score(ts_predictions, nn_predictions))

gp_train_accuracy_mean = np.round(np.mean(gp_nn_tsl_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_nn_tsl_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_nn_tsl_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_nn_tsl_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_nn_tsl_r2s),3)
gp_r2_std = np.round(np.std(gp_nn_tsl_r2s),3)
gp_time_mean = np.round(np.mean(gp_nn_tsl_time),3)
gp_time_std = np.round(np.std(gp_nn_tsl_time),3)
nn_ts_r2_mean = np.round(np.mean(nn_tsl_r2s),3)
nn_ts_r2_std = np.round(np.std(nn_tsl_r2s),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")
print(f"TS NN R2: {nn_ts_r2_mean} ± {nn_ts_r2_std}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
GP Train RMSE: 291.717 ± 0.0
GP Test RMSE: 290.255 ± 0.0
GP R2: 0.144 ± 0.0
GP Time: 0.08 ± 0.011
TS NN R2: 0.217 ± 0.0


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [18]:
# GPR model
gp_nn_tsc_train_accuracy = []
gp_nn_tsc_test_accuracy = []
gp_nn_tsc_r2s = []
nn_tsc_r2s = []
gp_nn_tsc_time = []
gpr_nn_tsc_model = None

# From NN and ts constant
for fold in range(number_of_folds):
    
    kernel = 1.0 * Matern(length_scale=1.0, nu=1.5)
    gpr_nn_tsc_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9, normalize_y=False)

    start_time = time.time()
    inputs = tsc_model.cen
    ts_predictions = tsc_model.predict(tsc_model.cen + 1e-8)
    nn_predictions = nn_model.predict(inputs)
    gpr_nn_tsc_model.fit(inputs, nn_predictions)
    time_used = time.time() - start_time

    train_means = gpr_nn_tsc_model.predict(x_train)
    test_means = gpr_nn_tsc_model.predict(x_test)

    train_rmse = root_mean_squared_error(y_train, train_means)
    test_rmse = root_mean_squared_error(y_test, test_means)

    gp_nn_tsc_train_accuracy.append(train_rmse)
    gp_nn_tsc_test_accuracy.append(test_rmse)
    gp_r2 = r2_score(y_test, test_means)
    gp_nn_tsc_r2s.append(gp_r2)
    gp_nn_tsc_time.append(time_used)
    nn_tsc_r2s.append(r2_score(ts_predictions, nn_predictions))

gp_train_accuracy_mean = np.round(np.mean(gp_nn_tsc_train_accuracy), 3)
gp_train_accuracy_std = np.round(np.std(gp_nn_tsc_train_accuracy), 3)
gp_test_accuracy_mean = np.round(np.mean(gp_nn_tsc_test_accuracy), 3)
gp_test_accuracy_std = np.round(np.std(gp_nn_tsc_test_accuracy),3)
gp_r2_mean = np.round(np.mean(gp_nn_tsc_r2s),3)
gp_r2_std = np.round(np.std(gp_nn_tsc_r2s),3)
gp_time_mean = np.round(np.mean(gp_nn_tsc_time),3)
gp_time_std = np.round(np.std(gp_nn_tsc_time),3)
nn_ts_r2_mean = np.round(np.mean(nn_tsc_r2s),3)
nn_ts_r2_std = np.round(np.std(nn_tsc_r2s),3)

print(f"GP Train RMSE: {gp_train_accuracy_mean} ± {gp_train_accuracy_std}")
print(f"GP Test RMSE: {gp_test_accuracy_mean} ± {gp_test_accuracy_std}")
print(f"GP R2: {gp_r2_mean} ± {gp_r2_std}")
print(f"GP Time: {gp_time_mean} ± {gp_time_std}")
print(f"TS NN R2: {nn_ts_r2_mean} ± {nn_ts_r2_std}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
e:\Users\PPL\anaconda3\envs\pytorch\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GP Train RMSE: 290.683 ± 0.0
GP Test RMSE: 289.229 ± 0.0
GP R2: 0.15 ± 0.0
GP Time: 0.077 ± 0.009
TS NN R2: 0.21 ± 0.0
